In [1]:
!pip install mxnet==1.6.0 numpy==1.23.1
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install pytorch-transformers
!pip install nltk


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-puqcwcac/kobert-tokenizer_ef2ab81ca6344ed98a8b55a5720349d8
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-puqcwcac/kobert-tokenizer_ef2ab81ca6344ed98a8b55a5720349d8
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
Defaulting to 

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
EMBED_SIZE = 768
NUM_HEADS = 8
MAX_LEN = 256
BATCH = 8
LR_RATE=1e-4
EPOCHS = 10


/home/minhyuk/.local/lib/python3.8/site-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '
[nltk_data] Downloading package punkt to /home/minhyuk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
news_dict = dict()
for row in pd.read_csv('data/news/news_body/crypto_news_body.csv').itertuples():
    if type(row.body) is not str: # nan
        continue
    if row.time in news_dict:
        news_dict[row.time].append(row.body)
    else:
        news_dict[row.time] = [row.body]
print(news_dict['2022-01-04'])

label_dict = dict() # default value is zero
for row in pd.read_csv('data/marketCap/label/labeled_CRYPTO.csv').itertuples():
    if row.score > 0:
        label_dict[row.time] = 1
    else:
        label_dict[row.time] = 0
print(label_dict)

['The Ethereum Name Service (ENS), a protocol that sells nonfungible tokens (NFTs) of domains representing wallet addresses, generated buzz within the crypto community in November after it airdropped tokens to its users. Those who claimed the tokens earned governance rights over the ENS and can vote on future decisions regarding the protocol.It was so well received that it prompted other Ethereum projects to airdrop tokens too. Though airdrops aren\'t new, they\'ve recently become increasingly common.In an airdrop, projects distribute tokens to specific investors\' wallets. In some situations, like with the ENS, projects airdrop tokens to those who have used their product. Other times, projects airdrop tokens to potential investors in hopes of marketing their product.The process of receiving the airdropped tokens can differ too. Sometimes, investors must choose to accept the tokens by claiming them, while other times, investors cannot reject the airdrop and tokens are automatically dro

In [3]:
X = []
for date in news_dict.keys() & label_dict.keys():
    X.append({"raw" : news_dict[date], "date":date, "label":label_dict[date]})
print(X[0]['raw'])
print(X[0]['date'])
print(X[0]['label'])


['CoinFlex said it would issue $47 million of a digital coin it is calling Recovery Value USD, or rvUSD.CoinFlex CEO Mark Lamb said in a blogpost that a long-time customer\'s account went into "negative equity." That prompted the company to halt withdrawals.By issuing the new rvUSD tokens, CoinFlex will be hoping to raise enough money to cover the shortfall in its books left by the investor and resume withdrawals for users. Cryptocurrency exchange CoinFlex on Tuesday issued a new token to raise funds in a bid to restart withdrawals for its customers, after one client failed to repay a massive debt.CoinFlex said it would issue $47 million worth of a digital coin, offering 20% interest, which it\'s calling Recovery Value USD, or rvUSD.It comes after the company paused withdrawals for customers last week citing "extreme market conditions" and "uncertainty involving a counterparty."On Monday, CoinFlex published a blogpost with more details about the counterparty. CEO Mark Lamb said in the 

In [4]:
import re

def add_spaces(text):
    # Pattern to match concatenated sentences
    pattern = r'(?<!\d)(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.)(?!\d)(?!\w\.\w)(?![A-Z][a-z]\.)'
    # Add space after period using the pattern
    spaced_text = re.sub(pattern, ' ', text)
    return spaced_text

In [5]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
for i, news_group in enumerate(X):
    news_group['sentences'] = []
    for j, news in enumerate(news_group['raw']):
        news_group['sentences'].append(tokenizer.tokenize(add_spaces(news)))

In [6]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert = BertModel.from_pretrained('bert-base-cased').to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
for i, news_group in enumerate(X):
    news_group['sentences_encoding'] = []
    for j, sentences in enumerate(news_group['sentences']):
        encodings = []
        for sentence in sentences:
            enc = bert_tokenizer.encode_plus(
                  sentence,
                  add_special_tokens=True,
                  max_length=MAX_LEN,
                  return_token_type_ids=False,
                  padding='max_length',
                  return_tensors='np',
                  ).input_ids
            emb = bert.forward(torch.from_numpy(enc).to(device)).pooler_output.cpu().detach().numpy()
            encodings.append(emb[0])
        news_group['sentences_encoding'].append(encodings)

In [9]:
X[0]['sentences'][1][4] # 2022-01-04의 두번째 뉴스의 다섯번째 문장

'The biggest misunderstanding around cryptocurrencies is that if they\'re "not doing a great job of replacing the U. S. dollar, then crypto is failing in its mission," Brooks, now the CEO of bitcoin mining and crypto tech company Bitfury Group, told CNBC\'s Ylan Muii at the Aspen Ideas Festival on Monday.'

In [10]:
X[0]['sentences_encoding'][1][4] # 2022-01-04의 두번째 뉴스의 다섯번째 문장의 bert 인코딩

array([-0.3909702 ,  0.34632587,  0.99769986, -0.9659785 ,  0.74896276,
        0.9745248 ,  0.7689903 , -0.99787337, -0.8929932 , -0.09361592,
        0.9055611 ,  0.988679  , -0.99946934, -0.99794024,  0.8423971 ,
       -0.79562414,  0.92764467, -0.32809824, -0.9992723 , -0.36564717,
       -0.75890803, -0.99804544,  0.10416395,  0.98492634,  0.72827303,
        0.06069084,  0.919355  ,  0.9995566 ,  0.3971513 , -0.47717744,
        0.04779362, -0.9486141 ,  0.91673493, -0.98896897,  0.23323989,
        0.5184841 ,  0.8463731 , -0.09684449,  0.6734623 , -0.94865507,
       -0.07568976, -0.52607435,  0.718838  , -0.3401398 ,  0.95755965,
       -0.15001586, -0.01475518, -0.09948013,  0.07558376,  0.9933939 ,
       -0.5712447 , -0.7070382 , -0.9977708 ,  0.5533274 ,  0.9668734 ,
        0.12889165,  0.9696663 , -0.0110752 , -0.99971086, -0.46809107,
        0.8919303 ,  0.4039762 ,  0.5826248 ,  0.32646158,  0.16087152,
       -0.23569421, -0.9433185 ,  0.22570778, -0.24630225,  0.05

In [11]:
multihead_attn = nn.MultiheadAttention(EMBED_SIZE, NUM_HEADS).to(device)

In [12]:
for i, news_group in enumerate(X):
    news_embeddings = []
    for j, sentences_encoding in enumerate(news_group['sentences_encoding']):
        query = key = value = torch.tensor(np.array(sentences_encoding)).to(device)
        attn_output, attn_output_weights = multihead_attn(query, key, value)
        news_embeddings.append(attn_output.mean(dim=0).cpu().detach().numpy())
    news_group['news_embeddings'] = news_embeddings

In [13]:
for i, news_group in enumerate(X):
    query = key = value = torch.tensor(np.array(news_group['news_embeddings'])).to(device)
    attn_output, attn_output_weights = multihead_attn(query, key, value)
    news_group['date_embedding'] = attn_output.mean(dim=0).detach().cpu().numpy()

In [14]:
X[0]['date_embedding'] # 2022-01-04의 임베딩

array([ 4.05694321e-02, -2.32308254e-01, -2.43725963e-02,  1.56033680e-01,
        4.13340777e-02, -1.40239492e-01, -3.07808127e-02,  7.52342632e-03,
       -1.46271348e-01,  8.12880024e-02,  1.52813476e-02,  5.48344105e-03,
        6.98007047e-02,  1.84107460e-02,  6.54059201e-02, -1.50618717e-01,
       -1.68354139e-01, -3.90793160e-02, -1.42386436e-01, -2.94691324e-01,
       -2.17363402e-01, -1.01280391e-01,  1.24899499e-01, -3.91151197e-02,
        6.43899143e-02,  1.16752379e-01,  6.59342259e-02,  2.05204293e-01,
        9.58002210e-02, -1.04738317e-01,  1.32651813e-03,  2.33170204e-02,
       -9.43399295e-02,  4.86717671e-02, -6.75244629e-02,  1.21288337e-02,
        2.97673233e-03, -5.37108909e-03,  7.62194842e-02, -4.66395169e-04,
        9.15888697e-02, -9.22254287e-03, -1.99614584e-01,  5.62423393e-02,
        1.64805457e-01,  1.57257944e-01, -3.27078663e-02,  9.16227102e-02,
        2.16402516e-01, -2.27462262e-01,  1.31599292e-01,  3.06630414e-02,
       -3.85431349e-02,  

In [15]:
import pickle
with open('data.pkl', 'wb') as f:
    pickle.dump(X, f)

# with open('data.pkl', 'rb') as f:
#     X = pickle.load(f)

In [16]:
class NewsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        date_emb = torch.from_numpy(self.data[idx]["date_embedding"])
        label = torch.tensor(self.data[idx]["label"])
        return date_emb, label

class FFN(nn.Module):
  def __init__(self, input_size, n_classes, dropout=0.5):
    super(FFN, self).__init__()
    self.layers = nn.Sequential(
        nn.Linear(input_size, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(p=dropout),
        nn.Linear(256, n_classes),
    )
  def forward(self, _input):
    return self.layers(_input)
  
def create_data_loader(dataset):
  return DataLoader(
            dataset,
            batch_size=BATCH,
            num_workers=2
        )

def train_epoch1(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model.train()
  losses = []
  correct_predictions = 0
  for batch in tqdm(data_loader):
    data, targets = batch
    data, targets = data.to(device), targets.to(device)
    outputs = model(data)
    preds = torch.argmax(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model1(model, data_loader, loss_fn, device, n_examples):
  model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in tqdm(data_loader):
      data, targets = batch
      data, targets = data.to(device), targets.to(device)
      outputs = model(data)
      preds = torch.argmax(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [17]:
split = int(len(X) * 0.9)
train_data, test_data = X[:split], X[split:]
train_data_loader = create_data_loader(NewsDataset(train_data))
test_data_loader = create_data_loader(NewsDataset(test_data))
model = FFN(EMBED_SIZE, 2).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)
os.makedirs('./checkpoint', exist_ok=True)

In [18]:
best_accuracy = 0
print(">> Training Start")
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch1(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(train_data))
  print(f'Train loss {train_loss} accuracy {train_acc}')
  val_acc, val_loss = eval_model1(model, test_data_loader, loss_fn, device, len(test_data))
  print(f'Val loss {val_loss} accuracy {val_acc}')
  if best_accuracy < val_acc:
    best_accuracy = val_acc
    check = {
        'state_dict': model.state_dict(),
    }
    
    torch.save(check, f"./checkpoint/{epoch}.ckpt")

>> Training Start
Epoch 1/10
----------


100%|██████████| 29/29 [00:00<00:00, 83.55it/s] 


Train loss 0.7026787256372387 accuracy 0.5132743362831859


100%|██████████| 4/4 [00:00<00:00, 15.09it/s]


Val loss 0.6896548122167587 accuracy 0.6153846153846154
Epoch 2/10
----------


...

Val loss 0.6973901987075806 accuracy 0.42307692307692313
Epoch 10/10
----------


100%|██████████| 29/29 [00:00<00:00, 82.14it/s]


Train loss 0.6130277483627714 accuracy 0.6946902654867256


100%|██████████| 4/4 [00:00<00:00, 15.69it/s]

Val loss 0.7396221831440926 accuracy 0.42307692307692313
